# T8 - Immunity methods

This tutorial covers several of the features new to Covasim 3.0, including waning immunity, multi-variant modelling, and advanced vaccination methods.

## Using waning immunity

By default, infection is assumed to confer lifelong perfect immunity, meaning that people who have been infected cannot be infected again.
However, this can be changed by setting `use_waning=True` when initializing a simulation.
When `use_waning` is set to True, agents in the simulation are assigned an initial level of neutralizing antibodies after recovering from an infection, drawn from a distribution defined in the parameter dictionary.
This level decays over time, leading to declines in the efficacy of protection against infection, symptoms, and severe symptoms.
The following example creates simulations without waning immunity (the default), and compares it to simulations with different speeds of immunity waning.

In [ ]:
import numpy as np
import sciris as sc
import covasim as cv
cv.options.set(dpi=100, show=False, close=True, verbose=0) # Standard options for Jupyter notebook

# Create sims with and without waning immunity
sim_nowaning = cv.Sim(n_days=120, label='No waning immunity')
sim_waning = cv.Sim(use_waning=True, n_days=120, label='Waning immunity')

# Now create an alternative sim with faster decay for neutralizing antibodies
sim_fasterwaning = cv.Sim(
    label='Faster waning immunity',
    n_days=120,
    use_waning=True,
    nab_decay=dict(form='nab_decay', decay_rate1=np.log(2)/30, decay_time1=250, decay_rate2=0.001)
)


# Create a multisim, run, and plot results
msim = cv.MultiSim([sim_nowaning, sim_waning, sim_fasterwaning])
msim.run()
msim.plot()

## Multi-variant modeling

The next examples show how to introduce new variants into a simulation.
These can either be known variants of concern, or custom new variants.
New variants may have differing levels of transmissibility, symptomaticity, severity, and mortality.
When introducing new variants, `use_waning` must be set to `True`.
The model includes known information about the levels of cross-immunity between different variants.
Cross-immunity can also be manually adjusted, as illustrated in the example below.

In [ ]:
# Define three new variants: B117, B1351, and a custom-defined variant
b117    = cv.variant('b117', days=10, n_imports=10)
b1351   = cv.variant('b1351', days=10, n_imports=10)
custom  = cv.variant(label='3x more transmissible', variant={'rel_beta': 3.0}, days=20, n_imports=10)

# Create the simulation
sim = cv.Sim(use_waning=True, variants=[b117, b1351, custom])

# Run and plot
sim.run()
sim.plot('variant')

## Advanced vaccination methods

The intervention `cv.vaccinate()` allows you to introduce a selection of known vaccines into the model, each of which is pre-populated with known parameters on their efficacy against different variants, their durations of protection, and the levels of protection that they afford against infection and disease progression.
When using `cv.vaccinate()`, `use_waning` must be set to `True`.
The following example illustrates how to use the `cv.vaccinate()` intervention.

In [ ]:
# Create some base parameters
pars = {
    'beta': 0.015,
    'n_days': 120,
}

# Define a Pfizer vaccine
pfizer = cv.vaccinate(vaccine='pfizer', days=20)
sim = cv.Sim(
    use_waning=True,
    pars=pars,
    interventions=pfizer
)
sim.run()
sim.plot()
